In [9]:
import web3
from web3 import Web3
import csv
import math
from typing import Dict, List
from eth_abi import decode
from decimal import Decimal
import json

# =================== CONFIGURATION ======================
POOL_ADDRESS='0x88e6A0c2dDD26FEEb64F039a2c41296FcB3f5640'
RPC_ENDPOINT='http://localhost:8545'
CSV_FILENAME="WETH/USDC.csv"
START_BLOCK=1234578
END_BLOCK='latest'

w3 = Web3(Web3.HTTPProvider(RPC_ENDPOINT))
assert w3.is_connected(), "Failed to connect to node"

UNISWAP_V3_POOL_ABI = ""
with open(r'/ethnode/queries/src/abi/V3_ABI.json', 'r') as abi:
    UNISWAP_V3_POOL_ABI = json.load(abi)

ERC_20_ABI = ""
with open(r'/ethnode/queries/src/abi/ERC_ABI.json', 'r') as abi:
    ERC_20_ABI = json.load(abi)

KNOWN_DECIMALS = {
    # Mainnet addresses
    "0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48": 6,  # USDC
    "0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48": 18, # WETH
    # Add other tokens as needed
}

In [14]:
class ERC20:
    def __init__(self, address):
        self.address = Web3.to_checksum_address(address)
        self.contract = w3.eth.contract(address=address, abi=ERC_20_ABI)

    def decimals(self):
        if self.address in KNOWN_DECIMALS:
            return KNOWN_DECIMALS[self.address]
        try:
            return self.contract.functions.decimals().call()
        except: 
            return self._fetch_decimals()
        
    def _fetch_decimals(self):
        try:
            data = w3.keccak(text='decimals()')[:4]  # Fixed function signature
            result = w3.eth.call({
                'to': self.address,
                'data': data
            })
            return int.from_bytes(result, 'big')
        except:
            return 18  # Default to 18 decimals if fetching fails

    def symbol(self):
        try:
            return self.contract.functions.symbol().call()
        except:
            return "UNKNOWN"

def price_conversion(sqrtpricex96, token0, token1):
    price_ratio = (sqrtpricex96 / 2**96) ** 2
    decimals_factor = 10 ** (token0.decimals() - token1.decimals())
    return price_ratio * decimals_factor

def tvl_calculation(liquidity, sqrtpricex96, token0, token1):
    price = price_conversion(sqrtpricex96, token0, token1)

    try:
        tvl_token0 = liquidity / (10 ** token0.decimals())
        tvl_token1 = tvl_token0 * price

        # Optional: Approximate USD value if token price oracle is available
        tvl_usd = None  # You need an external price oracle for this

        return {
            'tvl_token0': tvl_token0,
            'tvl_token1': tvl_token1,
            'tvl_usd': tvl_usd  # Now included, even if it's None
        }
    except Exception as e:
        print(f"Error in TVL Calculation: {str(e)}")
        return {'tvl_token0': 0, 'tvl_token1': 0, 'tvl_usd': 0}

def get_pool_state(pool, block):
    try:
        slot0 = pool.functions.slot0().call(block_identifier=block)
        liquidity = pool.functions.liquidity().call(block_identifier=block)

        token0 = ERC20(pool.functions.token0().call())
        token1 = ERC20(pool.functions.token1().call())

        sqrt_price = slot0[0]
        price = price_conversion(sqrt_price, token0, token1)

        tvl = tvl_calculation(liquidity, sqrt_price, token0, token1)

        return {
            'block_number': block,
            'timestamp': w3.eth.get_block(block)['timestamp'],
            'sqrtPriceX96': sqrt_price,
            'price': price,
            'tick': slot0[1],
            'liquidity': liquidity,
            'tvl_token0': tvl['tvl_token0'],
            'tvl_token1': tvl['tvl_token1'],
            'tvl_usd': tvl['tvl_usd'],  # Now safely included
            'token0_symbol': token0.symbol(),
            'token1_symbol': token1.symbol()
        }
    except Exception as e:
        print(f"Error at block {block}: {str(e)}")
        return None


In [17]:
def main():
    pool =  w3.eth.contract(address=POOL_ADDRESS, abi=UNISWAP_V3_POOL_ABI)
    token0 = ERC20(pool.functions.token0().call())
    token1 = ERC20(pool.functions.token1().call())

    print(f'Analyzing pool {POOL_ADDRESS}.')
    print(f'Pair: {token0.symbol()}/{token1.symbol()}')

    latest_block = w3.eth.block_number
    target_block = max(latest_block - 100, 12_300_000)
    
    data = get_pool_state(pool, 22000000)


main()
    

Analyzing pool 0x88e6A0c2dDD26FEEb64F039a2c41296FcB3f5640.
Pair: USDC/WETH
Error at block 22000000: Could not decode contract function call to slot0() with return data: b'', output_types: ['uint160', 'int24', 'uint16', 'uint16', 'uint16', 'uint8', 'bool']
